In [34]:
import time
import futu as ft


class AS(object):
    # API parameter setting
    api_svr_ip = '127.0.0.1'  # 账户登录的牛牛客户端PC的IP, 本机默认为127.0.0.1
    api_svr_port = 11111  # 富途牛牛端口，默认为11111
    unlock_password = "112778"  # 美股和港股交易解锁密码
    trade_env = ft.TrdEnv.SIMULATE  # 交易环境
    order_type = ft.OrderType.NORMAL  # 订单类型
    def __init__(self, stock, buy_rate, sell_rate, unit):
        self.stock = stock
        self.buy_rate = buy_rate
        self.sell_rate = sell_rate
        self.unit = unit
        self.quote_ctx, self.trade_ctx = self.context_setting()
    
    def close(self):
        self.quote_ctx.close()
        self.trade_ctx.close()

    def context_setting(self):
        if self.unlock_password == "":
            raise Exception("请先配置交易解锁密码! password: {}".format(
                self.unlock_password))

        quote_ctx = ft.OpenQuoteContext(
            host=self.api_svr_ip, port=self.api_svr_port)

        if 'HK.' in self.stock:
            trade_ctx = ft.OpenHKTradeContext(host=self.api_svr_ip, port=self.api_svr_port)
        elif 'US.' in self.stock:
            trade_ctx = ft.OpenUSTradeContext(host=self.api_svr_ip, port=self.api_svr_port)
        else:
            raise Exception("不支持的stock: {}".format(self.stock))

        if self.trade_env == ft.TrdEnv.SIMULATE:
            ret_code, ret_data = trade_ctx.unlock_trade(
                self.unlock_password)
            if ret_code == ft.RET_OK:
                print('解锁交易成功!')
            else:
                raise Exception("请求交易解锁失败: {}".format(ret_data))
        else:
            print('解锁交易成功!')

        return quote_ctx, trade_ctx
    
    
    def handle_data(self):
        while 1==1:
            self.quote_ctx = ft.OpenQuoteContext(host='127.0.0.1', port=11111)
            handler = TickerTest()
            self.quote_ctx.set_handler(handler)  # 设置实时逐笔推送回调
            self.quote_ctx.subscribe([self.stock], [SubType.TICKER]) # 订阅逐笔类型，OpenD开始持续收到服务器的推送
            time.sleep(60)  # 设置脚本接收OpenD的推送持续时间为60秒
            self.quote_ctx.close()   # 关闭当条连接，OpenD会在1分钟后自动取消相应股票相应类型的订阅
 
if __name__ == "__main__":
    STOCK = "HK.09988"
    BUY_RATE = 12
    SELL_RATE = 26
    UNIT = 9


    test = AS(STOCK, BUY_RATE, SELL_RATE, UNIT)
    test.handle_data()
    test.close()
    



2020-10-08 19:38:39,790 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-10-08 19:38:39,795 [open_context_base.py] on_connected:344: Connected : conn_id=6419; 
2020-10-08 19:38:39,801 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=6419; info={'server_version': 217, 'login_user_id': 18486115, 'conn_id': 6719858518717820586, 'conn_key': '5BC64FB5B6919095', 'conn_iv': 'EC77917B74108EB3', 'keep_alive_interval': 10, 'is_encrypt': False};
2020-10-08 19:38:39,808 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-10-08 19:38:39,810 [open_context_base.py] on_connected:344: Connected : conn_id=6420; 
2020-10-08 19:38:39,815 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=6420; info={'server_version': 217, 'login_user_id': 18486115, 'conn_id': 6719858518779939128, 'conn_key': '4FCC4FB5B6913870', 'conn_iv': '82945E2775107EAC

解锁交易成功!


Exception in thread Thread-6425:
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.8/site-packages/futu/common/callback_executor.py", line 33, in run
    ctx.packet_callback(proto_id, rsp_pb)
  File "/Applications/anaconda3/lib/python3.8/site-packages/futu/common/open_context_base.py", line 428, in packet_callback
    handler_ctx.recv_func(rsp_pb, proto_id)
  File "/Applications/anaconda3/lib/python3.8/site-packages/futu/common/handler_context.py", line 89, in recv_func
    handler.on_recv_rsp(rsp_pb)
  File "<ipython-input-33-d1d66d9df5bd>", line 14, in on_recv_rsp
UnboundLocalError: local variable 'time' referenced before assignment


code  HK.09988
price  288.4


KeyboardInterrupt: 

In [3]:
from futu import *

class TickerTest(TickerHandlerBase):
    collection = []
    myStockList = []
    startTime = 0
    flag = true
    def on_recv_rsp(self, rsp_str):
        ret_code, data = super(TickerTest,self).on_recv_rsp(rsp_str)
        if ret_code != RET_OK:
            print("TickerTest: error, msg: %s" % data)
            return RET_ERROR, data
        print("code ", data['code'][0])
        print("price ", data['price'][0])
        print("time ", data['time'][0])
        timeArray = time.strptime(data['time'][0], "%Y-%m-%d %H:%M:%S")
        timeInmill = time.mktime(timeArray)
        print("time ", time.mktime(timeArray))
        unit = [data['code'][0],data['price'][0],timeInmill]
        self.myStockList.append(unit)
        if  flag:
            flag = false
            self.starttime = timeInmill
        diffTime = timeInmill - self.startTime
        print("timeInmill ", timeInmill)
        print("startTime ", self.startTime)
        print("dif time ", diffTime)
        if (timeInmill - self.startTime) >2:
            self.starttime = timeInmill
            tempList = self.myStockList
            self.collection.append(tempList)
            stockList = self.collection[len(self.collection)-1]
#             print("stockList", stockList)
            if len(stockList)>=2:
                self.calculateAcceleration(stockList)
        return RET_OK, data
    
    def calculateAcceleration(self,myStockList):
#         print("myStockList", myStockList)
        collength = len(myStockList)
        if collength>=1:
            myList =self.collection[len(myStockList)-1]
            acceleration = self.getAcceleration(myList)
            
    def getAcceleration(self,myList):
        lengthoflist=len(myList)
        if lengthoflist>=2:
#             print("myList", myList)
            stock1 = myList[0]
            stock2 = myList[lengthoflist-1]
            a =(stock1[1]-stock2[1])/(stock1[2]-stock2[2])
            print("acceleration ", a)
        return a
        

#get realtime 
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
handler = TickerTest()
quote_ctx.set_handler(handler)  # 设置实时逐笔推送回调
quote_ctx.subscribe(['HK.09988'], [SubType.TICKER]) # 订阅逐笔类型，OpenD开始持续收到服务器的推送
# time.sleep(60000)  # 设置脚本接收OpenD的推送持续时间为15秒
# quote_ctx.close()   # 关闭当条连接，OpenD会在1分钟后自动取消相应股票相应类型的订阅


code  HK.09988
price  289.6
time  2020-10-08 15:36:46
time  1602124606.0
timeInmill  1602124606.0
startTime  0
dif time  1602124606.0
acceleration  -0.0


NameError: name 'true' is not defined

code  HK.09988
price  289.4
time  2020-10-08 15:36:46
time  1602124606.0
timeInmill  1602124606.0
startTime  0
dif time  1602124606.0
acceleration  -0.0015873015873019481
code  HK.09988
price  289.4
time  2020-10-08 15:36:46
time  1602124606.0
timeInmill  1602124606.0
startTime  0
dif time  1602124606.0
acceleration  -0.0015873015873019481
code  HK.09988
price  289.6
time  2020-10-08 15:36:46
time  1602124606.0
timeInmill  1602124606.0
startTime  0
dif time  1602124606.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:36:51
time  1602124611.0
timeInmill  1602124611.0
startTime  0
dif time  1602124611.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:36:51
time  1602124611.0
timeInmill  1602124611.0
startTime  0
dif time  1602124611.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:36:56
time  1602124616.0
timeInmill  1602124616.0
startTime  0
dif time  1602124616.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 1

code  HK.09988
price  289.6
time  2020-10-08 15:37:48
time  1602124668.0
timeInmill  1602124668.0
startTime  0
dif time  1602124668.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:48
time  1602124668.0
timeInmill  1602124668.0
startTime  0
dif time  1602124668.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:48
time  1602124668.0
timeInmill  1602124668.0
startTime  0
dif time  1602124668.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:48
time  1602124668.0
timeInmill  1602124668.0
startTime  0
dif time  1602124668.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:48
time  1602124668.0
timeInmill  1602124668.0
startTime  0
dif time  1602124668.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:49
time  1602124669.0
timeInmill  1602124669.0
startTime  0
dif time  1602124669.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:37:49
time  1602124669.0
timeInmil

code  HK.09988
price  289.8
time  2020-10-08 15:38:46
time  1602124726.0
timeInmill  1602124726.0
startTime  0
dif time  1602124726.0
acceleration  0.0008130081300812546
code  HK.09988
price  289.6
time  2020-10-08 15:38:47
time  1602124727.0
timeInmill  1602124727.0
startTime  0
dif time  1602124727.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:38:47
time  1602124727.0
timeInmill  1602124727.0
startTime  0
dif time  1602124727.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:38:47
time  1602124727.0
timeInmill  1602124727.0
startTime  0
dif time  1602124727.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:38:48
time  1602124728.0
timeInmill  1602124728.0
startTime  0
dif time  1602124728.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:38:48
time  1602124728.0
timeInmill  1602124728.0
startTime  0
dif time  1602124728.0
acceleration  -0.0
code  HK.09988
price  289.6
time  2020-10-08 15:38:52
time  16021

In [41]:
import time 

print("time ", time.mktime(timeArray))




time  1602038808.0


In [45]:
myStockList = []
for x in range(7):
    unit = [x,x+1,x+2]
    myStockList.append(unit)
    print(myStockList)



[[0, 1, 2]]
[[0, 1, 2], [1, 2, 3]]
[[0, 1, 2], [1, 2, 3], [2, 3, 4]]
[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5]]
[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6]]
[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7]]
[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8]]
